Testing different endpoints, to see which is the best to store game_data for

In [1]:
from nba_api.stats.endpoints import (scoreboardv2,
                                     boxscoretraditionalv2,
                                     leaguegamelog)
import pandas as pd

In [2]:
scoreboard = scoreboardv2.ScoreboardV2().get_data_frames()[0]
scoreboard.head(3)

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TV_BROADCASTER_ABBREVIATION,AWAY_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,ARENA_NAME,WH_STATUS,WNBA_COMMISSIONER_FLAG
0,2026-01-04T00:00:00,1,0022500494,3,Final,20260104/DETCLE,1610612739,1610612765,2025,4,,None,FDSNOH,FDSNDT,None,Rocket Arena,1,0
1,2026-01-04T00:00:00,2,0022500495,3,Final,20260104/INDORL,1610612753,1610612754,2025,4,,None,FDSNFL,FDSNIN,None,Kia Center,1,0
2,2026-01-04T00:00:00,3,0022500496,3,Final,20260104/DENBKN,1610612751,1610612743,2025,4,,None,YES,ALT2,None,Barclays Center,1,0


In [3]:
boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id='0022100001').get_data_frames()[1]
boxscore.head()

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,FGM,FGA,FG_PCT,FG3M,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022100001,1610612749,Bucks,MIL,Milwaukee,240:00,48,105,0.457,17,...,13,41,54,25,8,9,7,19,127,23
1,0022100001,1610612751,Nets,BKN,Brooklyn,240:00,37,84,0.440,17,...,5,39,44,19,3,9,12,17,104,-23


In [20]:
games = leaguegamelog.LeagueGameLog(league_id='00', 
                                    season='2015-16', 
                                    season_type_all_star='Regular Season'
                                ).get_data_frames()[0]

games.iloc[:, :7].sort_values(by="GAME_ID", ascending=True).head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP
4,22015,1610612765,DET,Detroit Pistons,0021500001,2015-10-27,DET @ ATL
5,22015,1610612737,ATL,Atlanta Hawks,0021500001,2015-10-27,ATL vs. DET
0,22015,1610612739,CLE,Cleveland Cavaliers,0021500002,2015-10-27,CLE @ CHI
2,22015,1610612741,CHI,Chicago Bulls,0021500002,2015-10-27,CHI vs. CLE
1,22015,1610612740,NOP,New Orleans Pelicans,0021500003,2015-10-27,NOP @ GSW


leaguegamelog is promising - update to db

In [5]:
import os
from dotenv import load_dotenv
from sqlalchemy import (create_engine, text)

In [6]:
load_dotenv()

engine = create_engine(
    f"postgresql+psycopg2://"
    f"{os.environ['USER']}:{os.environ['PASSWORD']}"
    f"@{os.environ['HOST']}:{os.environ['PORT']}"
    f"/{os.environ['DB']}",
    pool_pre_ping=True
)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM players;"))
    
    result = pd.DataFrame(result).head()

    conn.close()
    
result

,player_id,full_name,first_name,last_name,is_active,birthdate
0,2,Byron Scott,Byron,Scott,False,1961-03-28
1,3,Grant Long,Grant,Long,False,1966-03-12
2,7,Dan Schayes,Dan,Schayes,False,1959-05-10
3,9,Sedale Threatt,Sedale,Threatt,False,1961-09-10
4,12,Chris King,Chris,King,False,1969-07-24
